In [2]:
import scanpy as sc
import scirpy as ir
import pandas as pd
import awkward as ak
# from sklearn.base import ClassNamePrefixFeaturesOutMixin

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
adata = sc.read_h5ad("../datasets/02_dex_annotated_cd8.h5ad")
adata

AnnData object with n_obs × n_vars = 53907 × 18471
    obs: 'is_cell', 'high_confidence', 'multi_chain', 'extra_chains', 'IR_VJ_1_c_call', 'IR_VJ_2_c_call', 'IR_VDJ_1_c_call', 'IR_VDJ_2_c_call', 'IR_VJ_1_consensus_count', 'IR_VJ_2_consensus_count', 'IR_VDJ_1_consensus_count', 'IR_VDJ_2_consensus_count', 'IR_VJ_1_d_call', 'IR_VJ_2_d_call', 'IR_VDJ_1_d_call', 'IR_VDJ_2_d_call', 'IR_VJ_1_duplicate_count', 'IR_VJ_2_duplicate_count', 'IR_VDJ_1_duplicate_count', 'IR_VDJ_2_duplicate_count', 'IR_VJ_1_j_call', 'IR_VJ_2_j_call', 'IR_VDJ_1_j_call', 'IR_VDJ_2_j_call', 'IR_VJ_1_junction', 'IR_VJ_2_junction', 'IR_VDJ_1_junction', 'IR_VDJ_2_junction', 'IR_VJ_1_junction_aa', 'IR_VJ_2_junction_aa', 'IR_VDJ_1_junction_aa', 'IR_VDJ_2_junction_aa', 'IR_VJ_1_locus', 'IR_VJ_2_locus', 'IR_VDJ_1_locus', 'IR_VDJ_2_locus', 'IR_VJ_1_productive', 'IR_VJ_2_productive', 'IR_VDJ_1_productive', 'IR_VDJ_2_productive', 'IR_VJ_1_v_call', 'IR_VJ_2_v_call', 'IR_VDJ_1_v_call', 'IR_VDJ_2_v_call', 'has_ir', 'LTDEMIAQY', 'YTN

In [3]:
# selected_patients = ['A01', 'A15']
# adata = adata[adata.obs['donor'].isin(selected_patients)].copy()

In [4]:
print(adata.X[0:4,11:20].toarray())
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
print(adata.X[0:4,11:20].toarray())

[[2.4700055 0.        0.        0.        0.        0.        0.
  1.3098526 0.       ]
 [0.        0.        0.        0.        0.        0.        0.
  0.        0.       ]
 [3.1313903 0.        0.        0.        0.        0.        0.
  0.        0.       ]
 [0.        0.        0.        0.        0.        0.        0.
  0.        0.       ]]
[[2.417449  0.        0.        0.        0.        0.        0.
  1.8591411 0.       ]
 [0.        0.        0.        0.        0.        0.        0.
  0.        0.       ]
 [2.4150453 0.        0.        0.        0.        0.        0.
  0.        0.       ]
 [0.        0.        0.        0.        0.        0.        0.
  0.        0.       ]]


In [5]:
print('Shape before: ', adata.shape)
sc.pp.highly_variable_genes(adata, n_top_genes=5000)
adata = adata[:, adata.var['highly_variable']]
print('Shape after: ', adata.shape)

Shape before:  (53907, 18471)
Shape after:  (53907, 5000)


In [6]:
ir.io.upgrade_schema(adata)

/Users/tristanaretz/mambaforge/envs/vaccineSimulator/lib/python3.10/site-packages/anndata/utils.py:334: ExperimentalFeatureWarning: Support for Awkward Arrays is currently experimental. Behavior may change in the future. Please report any issues you may encounter!
  warnings.warn(msg, category, stacklevel=stacklevel)
/Users/tristanaretz/mambaforge/envs/vaccineSimulator/lib/python3.10/site-packages/scirpy/io/_legacy.py:39: ImplicitModificationWarning: Setting element `.obsm['airr']` of view, initializing view as actual.
  adata.obsm["airr"] = tmp_adata.obsm["airr"]


In [7]:
ir.pp.index_chains(adata)

100%|██████████| 11/11 [00:03<00:00,  3.23it/s]


In [8]:
adata.obsm['airr']

<Array [[{c_call: 'TRAC', ...}, ...], ...] type='53907 * var * {c_call: ?st...'>

In [9]:
print(len(adata))
junction_aa = ir.get.airr(adata, "junction_aa")
adata = adata[~(junction_aa['VJ_1_junction_aa'].isna() | junction_aa['VDJ_1_junction_aa'].isna())]
print(len(adata))

53907
44310


In [25]:
from mvtcr.utils_preprocessing import Preprocessing

In [26]:
adata

AnnData object with n_obs × n_vars = 44310 × 5000
    obs: '%_max_dextramer', 'ATDSLNNEY', 'C0251_A02', 'C0251_A08', 'C0252_A03', 'C0252_A04', 'C0253_A04', 'C0253_A05', 'C0254_A05', 'C0254_A11', 'C0255_A06', 'C0255_A15', 'C0256_A11', 'C0256_A29', 'C0257_A15', 'C0257_A19', 'C0258_A25', 'C0258_A29', 'CCR7', 'CD4 NV/CM_score', 'CD4/CD8_score', 'CD45RA', 'CD62L', 'CD8 Cytokine_score', 'CD8 Cytotoxic_score', 'CTELKLSDY', 'CXCR3', 'FLRGRAYGL', 'FPQSAPHGV', 'GTHWFVTQR', 'HA1', 'HA2', 'HA3', 'HA4', 'HA5', 'HA6', 'HA7', 'HA8', 'Hu.CD101', 'Hu.CD103', 'Hu.CD105_43A3', 'Hu.CD107a', 'Hu.CD112', 'Hu.CD119', 'Hu.CD11a', 'Hu.CD11b', 'Hu.CD11c', 'Hu.CD122', 'Hu.CD123', 'Hu.CD124', 'Hu.CD127', 'Hu.CD13', 'Hu.CD134', 'Hu.CD137', 'Hu.CD141', 'Hu.CD146', 'Hu.CD14_M5E2', 'Hu.CD152', 'Hu.CD154', 'Hu.CD155', 'Hu.CD158', 'Hu.CD158b', 'Hu.CD158e1', 'Hu.CD16', 'Hu.CD161', 'Hu.CD163', 'Hu.CD169', 'Hu.CD18', 'Hu.CD183', 'Hu.CD185', 'Hu.CD19', 'Hu.CD194', 'Hu.CD195', 'Hu.CD196', 'Hu.CD1c', 'Hu.CD1d', 'Hu.CD2', 'Hu

In [27]:
Preprocessing.preprocessing_pipeline(adata, 
                                     clonotype_key_added='clonotype', 
                                     airr_name='junction_aa',
                                     cond_vars=['donor'])

100%|██████████| 13715/13715 [00:07<00:00, 1921.46it/s]


In [28]:
adata

AnnData object with n_obs × n_vars = 44310 × 5000
    obs: '%_max_dextramer', 'ATDSLNNEY', 'C0251_A02', 'C0251_A08', 'C0252_A03', 'C0252_A04', 'C0253_A04', 'C0253_A05', 'C0254_A05', 'C0254_A11', 'C0255_A06', 'C0255_A15', 'C0256_A11', 'C0256_A29', 'C0257_A15', 'C0257_A19', 'C0258_A25', 'C0258_A29', 'CCR7', 'CD4 NV/CM_score', 'CD4/CD8_score', 'CD45RA', 'CD62L', 'CD8 Cytokine_score', 'CD8 Cytotoxic_score', 'CTELKLSDY', 'CXCR3', 'FLRGRAYGL', 'FPQSAPHGV', 'GTHWFVTQR', 'HA1', 'HA2', 'HA3', 'HA4', 'HA5', 'HA6', 'HA7', 'HA8', 'Hu.CD101', 'Hu.CD103', 'Hu.CD105_43A3', 'Hu.CD107a', 'Hu.CD112', 'Hu.CD119', 'Hu.CD11a', 'Hu.CD11b', 'Hu.CD11c', 'Hu.CD122', 'Hu.CD123', 'Hu.CD124', 'Hu.CD127', 'Hu.CD13', 'Hu.CD134', 'Hu.CD137', 'Hu.CD141', 'Hu.CD146', 'Hu.CD14_M5E2', 'Hu.CD152', 'Hu.CD154', 'Hu.CD155', 'Hu.CD158', 'Hu.CD158b', 'Hu.CD158e1', 'Hu.CD16', 'Hu.CD161', 'Hu.CD163', 'Hu.CD169', 'Hu.CD18', 'Hu.CD183', 'Hu.CD185', 'Hu.CD19', 'Hu.CD194', 'Hu.CD195', 'Hu.CD196', 'Hu.CD1c', 'Hu.CD1d', 'Hu.CD2', 'Hu

In [ ]:
# Alternatively group splitting is available by itself with:

# train, val = Preprocessing.group_shuffle_split(adata, group_col='clonotype', test_size=0.2, random_seed=42)

# adata.obs['set'] = 'train'
# adata.obs.loc[val, 'set'] = 'val'

# adata.obs["set"].value_counts()

In [31]:
path_out = '../datasets/02_dex_annotated_cd8_new.h5ad'
adata.write_h5ad(path_out, compression='gzip')